# Optimización y Procesos Estocásticos

## Análisis de Texto

## Ejemplo

### Cargar el dataset 20 newsgroups

In [1]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

Ahora se cargan las categorias seleccionadas

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

es posible imprimir el nombre de las categorias

In [3]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

los datos están almacenados en el atributo `data`

In [4]:
len(twenty_train.data)

2257

In [5]:
len(twenty_train.filenames)

2257

imprimir las 3 primeras líneas del primer registro

In [6]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


y el nombre de la categoría a la que pertenece

In [7]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


los algoritmos de aprendizaje supervisado requieren una etiqueta de categoría para documento en el conjunto de entrenamiento. 

Para mayor eficiencia (velocidad de acceso, almacenamiento), las categorias se guardan como un arregle de índices del nombre de la categoría en la lista `target_names`

In [8]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

es posible obtener el nombre de las categorias por medio de

In [9]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


### Extraer características de los archivos de texto

Para aplicar aprencizaje de máquina se debe convertir el contenido de texto en vectores de catacterísticas numéricas

#### Bolsa de palabras (Bag of words)

La bolsa de palabras es una forma intuitiva de representar vectores de características:

1. Asignar un entero a cada palabra presente en el documento del conjunto de entrenamiento (Ej: crear diccionario)

1. Para cada documento `#i`, contar el número de ocurrenciasde cada palabra `w` y almacenarla en `X[i, j]` como el valor de la característica `#j`, donde `j` es el índice de la palabra `w` en el diccionario.

#### Tokenizar texto

Procesar de texto, tokenizar y filtrar palabras son funciones de incluidas  en un componente que permite construir el diccionario de características.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

`CountVetorizer()` soporta el conteo de N-gramas de palabras o caracteres consecutivos. Una vez ajustado, el vectorizador construye un diccionario de índices de características.

In [11]:
count_vect.vocabulary_.get(u'algorithm')

4690

el índice de una palabra en el vocabulario está relacionado con su frecuencia en el corpus de entrenamiento. 

#### Ocurrencias y frecuencias

Conteo de ocurrencias es un buen inicio pero presenta un problema: los documentos grandes tendrán promedios mayores de conteos en relación con documentos más cortos; incluso así hablen acerca del mismo tema.

Para evitar esto se divide entre el número total de palabras. Esta característica es llamada *Frecuencia de Término* `tr`.

Otro refinamiento de `tr` consite en reducir la escala de los pesos de las palabras presentes en varios documentos y son por lo tanto menos informativas que equellas que ocurren una porción pequeña del corpus.

La reducción de escala es llamada *Frecuencia de Término – Frecuencia Inversa de Documento* `tr-idf`.

Para calcular `tr` y `tr-idf` se utiliza:

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [14]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

### Entrenar un clasificador

A partir de las características, es posible entrenar un clasificador para predecir la categoría de un post. Para iniciar se utilizará el clasificador *Bayesiano ingenuo*:

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

Para predecir la salida de un documento nuevo, se debe extraer el vector de características usando la misma secuencia de comandos: 

In [17]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


## Referencias

* http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html